In [1]:
import sys
sys.executable

'C:\\Users\\DELL\\Downloads\\ItineraireVacances\\.venv\\Scripts\\python.exe'

# Dataset — tripadvisor_european_restaurants
(679.68 MB, 1 file, 42 columns, 5774 downloads, Updated 5 years ago)


https://www.kaggle.com/datasets/stefanoleone992/tripadvisor-european-restaurants

In [2]:
import pandas as pd
import numpy as np
import pyarrow
from IPython.display import display
import folium
from folium.plugins import MarkerCluster

# 1. Charger les données

In [3]:
#lire CSV_PATH (première fois)
CSV_PATH = "tripadvisor_european_restaurants.csv"
df_trip = pd.read_csv(CSV_PATH, sep=',', encoding='utf-8', low_memory = False)

#Convertir en Parquet --> Fichier beaucoup plus léger --> Lecture rapide
PARQUET_PATH = "tripadvisor_european_restaurants.parquet"
df_trip.to_parquet(PARQUET_PATH, compression = "snappy")

#Libère la mémoire et recharger le parquet
del df_trip
df_trip = pd.read_parquet("tripadvisor_european_restaurants.parquet")

#Afficher 2 lignes et TOUTES les colonnes
pd.set_option("display.max_columns", None)
display(df_trip.head(2))

print("\n-------------------------------------")
print("Nombre de lignes et de colones")
print(df_trip.shape)

print("\n-------------------------------------")
print("Display info sur les colonnes et datatype")
display(df_trip.info())

print("\n-------------------------------------")
print("Lister + count les pays de la colonne country")
display(df_trip["country"].value_counts())

print("\n-------------------------------------")
print("Lister + count les régions en France")
display(df_trip.loc[df_trip["country"] == "France", "region"].value_counts())

,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,claimed,awards,popularity_detailed,popularity_generic,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,default_language,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,g10001637-d10002227,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131,Claimed,None,#1 of 2 Restaurants in Saint-Jouvent,#1 of 2 places to eat in Saint-Jouvent,"Cheap Eats, French",€,None,"Lunch, Dinner",French,None,"Reservations, Seating, Wheelchair Accessible, ...",N,N,N,None,NaN,NaN,NaN,4.0,36.0,English,2.0,2.0,0.0,0.0,0.0,0.0,4.0,4.5,4.0,NaN,None
1,g10001637-d14975787,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480,Unclaimed,None,#2 of 2 Restaurants in Saint-Jouvent,#2 of 2 places to eat in Saint-Jouvent,Cheap Eats,€,None,None,None,None,None,N,N,N,None,NaN,NaN,NaN,4.0,5.0,All languages,5.0,2.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,None



-------------------------------------
Nombre de lignes et de colones
(1083397, 42)

-------------------------------------
Display info sur les colonnes et datatype
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083397 entries, 0 to 1083396
Data columns (total 42 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   restaurant_link                    1083397 non-null  object 
 1   restaurant_name                    1083396 non-null  object 
 2   original_location                  1083397 non-null  object 
 3   country                            1083397 non-null  object 
 4   region                             1033074 non-null  object 
 5   province                           742765 non-null   object 
 6   city                               682712 non-null   object 
 7   address                            1083397 non-null  object 
 8   latitude                           1067607 non-null  floa

None


-------------------------------------
Lister + count les pays de la colonne country


country
Italy               224763
Spain               157479
France              155288
England             144681
Germany             115333
Greece               33763
Portugal             32592
The Netherlands      29792
Poland               24698
Belgium              23711
Austria              20487
Sweden               18555
Czech Republic       14844
Scotland             14215
Ireland              11203
Denmark               9485
Wales                 9134
Croatia               8375
Romania               7842
Hungary               7431
Finland               7372
Bulgaria              4469
Slovakia              4251
Northern Ireland      3634
Name: count, dtype: int64


-------------------------------------
Lister + count les régions en France


region
Ile-de-France                 31271
Auvergne-Rhone-Alpes          20753
Provence-Alpes-Cote d'Azur    19925
Occitanie                     17775
Nouvelle-Aquitaine            15542
Grand Est                      9885
Hauts-de-France                7889
Brittany                       7568
Pays de la Loire               6527
Normandy                       6096
Bourgogne-Franche-Comte        5290
Centre-Val de Loire            4230
Corsica                        2535
Name: count, dtype: int64

# 2. Décrire les données  (région Ile-de-France)

### 2.1. Extraire les données de la région Ile-de-France

In [4]:
#créer un DataFrame filtré avec country = "France" ET region = "Ile-de-France"
df_trip_idf = df_trip[(df_trip["country"] == "France") & (df_trip["region"] == "Ile-de-France")].copy()
print("Nombre de lignes et de colones")
print(df_trip_idf.shape, "\n")
df_trip_idf.head(3)

Nombre de lignes et de colones
(31271, 42) 



,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,claimed,awards,popularity_detailed,popularity_generic,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,default_language,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
74,g1006520-d1007522,L'imprevu,"[""Europe"", ""France"", ""Ile-de-France"", ""Yveline...",France,Ile-de-France,Yvelines,Conflans Sainte Honorine,"9 quai de la Republique, 78700 Conflans Sainte...",48.991110,2.088753,Claimed,"Travellers' Choice, Certificate of Excellence ...",#10 of 64 Restaurants in Conflans Sainte Honorine,#10 of 68 places to eat in Conflans Sainte Hon...,"Fine Dining, French",€€€€,None,"Lunch, Dinner, After-hours",French,None,"Reservations, Seating, Serves Alcohol, Accepts...",N,N,N,"{""Mon"": [""10:00-15:00"", ""18:00-23:00""], ""Tue"":...",7.0,60.0,13.0,4.0,292.0,English,2.0,1.0,1.0,0.0,0.0,0.0,4.0,4.0,3.5,3.0,None
75,g1006520-d10128445,La Maison du Poulet,"[""Europe"", ""France"", ""Ile-de-France"", ""Yveline...",France,Ile-de-France,Yvelines,Conflans Sainte Honorine,"91 avenue Carnot, 78700 Conflans Sainte Honori...",49.000570,2.093569,Claimed,"Travellers' Choice, Certificate of Excellence ...",#3 of 64 Restaurants in Conflans Sainte Honorine,#3 of 68 places to eat in Conflans Sainte Hono...,"Mid-range, French, Vegetarian Friendly",€€-€€€,None,"Lunch, Dinner",French,Vegetarian Friendly,None,Y,N,N,"{""Mon"": [], ""Tue"": [], ""Wed"": [""12:00-14:00"", ...",5.0,20.0,9.0,4.5,296.0,All languages,296.0,202.0,73.0,18.0,1.0,2.0,4.5,4.5,4.5,NaN,None
76,g1006520-d1017687,Little Tokyo,"[""Europe"", ""France"", ""Ile-de-France"", ""Yveline...",France,Ile-de-France,Yvelines,Conflans Sainte Honorine,"2 boulevard Troussel, 78700 Conflans Sainte Ho...",48.990112,2.082339,Unclaimed,"Certificate of Excellence 2016, Certificate of...",#13 of 64 Restaurants in Conflans Sainte Honorine,#13 of 68 places to eat in Conflans Sainte Hon...,"Mid-range, Japanese, Sushi, Asian",€€-€€€,None,"Lunch, Dinner","Japanese, Sushi, Asian",Vegetarian Friendly,None,Y,N,N,"{""Mon"": [""11:30-02:30""], ""Tue"": [""11:30-02:30""...",7.0,105.0,7.0,4.0,204.0,English,6.0,3.0,2.0,1.0,0.0,0.0,4.0,4.0,3.5,3.5,None


### 2.2. Les colonnes Dtype float64

In [5]:
print("\n-------------------------------------")
print("Describe la colonne open_days_per_week (disponibilité globale):")
print("---------------------------------------")
print("ex : 7 = ouvert tous les jours")
display(df_trip_idf["open_days_per_week"].unique())
display(df_trip_idf["open_days_per_week"].describe())

print("\n-------------------------------------")
print("Describe la colonne working_shifts_per_week (flexibilité horaire) :")
print("ex : 6 jours ouverts * 2 services/jours = 12 working shifts")
print("---------------------------------------")
display(df_trip_idf["working_shifts_per_week"].unique())
display(df_trip_idf["working_shifts_per_week"].describe())

print("\n-------------------------------------")
print("Describe la colonne avg_rating (perception globale sur TripAdvisor (1 à 5)) :")
print("---------------------------------------")
display(df_trip_idf["avg_rating"].unique())
display(df_trip_idf["avg_rating"].describe())

print("\n-------------------------------------")
print("Describe la colonne total_reviews_count (nombre total d’avis) (crédibilité de la note ---> notoriété / fréquentation) :")
print("---------------------------------------")
display(df_trip_idf["total_reviews_count"].unique())
display(df_trip_idf["total_reviews_count"].describe())

print("\n-------------------------------------")
print("Describe la colonne reviews_count_in_default_language (nombre d’avis dans la langue principale) :")
print("---------------------------------------")
display(df_trip_idf["reviews_count_in_default_language"].unique()[:10])
display(df_trip_idf["reviews_count_in_default_language"].describe())

print("\n-------------------------------------")
print("Describe la colonne excellent (nombre d’avis par catégorie excellent) :")
print("---------------------------------------")
display(df_trip_idf["excellent"].unique()[:10])
display(df_trip_idf["excellent"].describe())

print("\n-------------------------------------")
print("Describe la colonne very_good (nombre d’avis par catégorie very_good) :")
print("---------------------------------------")
display(df_trip_idf["very_good"].unique()[:10])
display(df_trip_idf["very_good"].describe())

print("\n-------------------------------------")
print("Describe la colonne average (nombre d’avis par catégorie average) :")
print("---------------------------------------")
display(df_trip_idf["average"].unique()[:10])
display(df_trip_idf["average"].describe())

print("\n-------------------------------------")
print("Describe la colonne poor (nombre d’avis par catégorie poor):")
print("---------------------------------------")
display(df_trip_idf["poor"].unique()[:10])
display(df_trip_idf["poor"].describe())

print("\n-------------------------------------")
print("Describe la colonne terrible (nombre d’avis par catégorie terrible):")
print("---------------------------------------")
display(df_trip_idf["terrible"].unique()[:10])
display(df_trip_idf["terrible"].describe())

print("\n-------------------------------------")
print("Describe la colonne food (notes spécifiques (1 à 5) sur food) :")
print("---------------------------------------")
display(df_trip_idf["food"].unique())
display(df_trip_idf["food"].describe())

print("\n-------------------------------------")
print("Describe la colonne service (notes spécifiques (1 à 5) sur service) :")
print("---------------------------------------")
display(df_trip_idf["service"].unique())
display(df_trip_idf["service"].describe())

print("\n-------------------------------------")
print("Describe la colonne value (notes spécifiques (1 à 5) sur value) :")
print("---------------------------------------")
display(df_trip_idf["value"].unique())
display(df_trip_idf["value"].describe())

print("\n-------------------------------------")
print("Describe la colonne atmosphere (notes spécifiques (1 à 5) sur atmosphere) :")
print("---------------------------------------")
display(df_trip_idf["atmosphere"].unique())
display(df_trip_idf["atmosphere"].describe())

#Vérifier les valeurs manquantes des colonnes Dtype float64
print("\n-------------------------------------")
print("Les valeurs manquantes des colonnes Dtype float64 (sur 31271 lignes) :")
print("---------------------------------------")
missing_float64_summary = pd.DataFrame({"missing_count": df_trip_idf.select_dtypes(include="float64").isna().sum(),
                                        "missing_pct": (df_trip_idf.select_dtypes(include="float64").isna().sum() / 31271 * 100).round(2)}
                                      ).sort_values("missing_pct", ascending=False)
display(missing_float64_summary)


-------------------------------------
Describe la colonne open_days_per_week (disponibilité globale):
---------------------------------------
ex : 7 = ouvert tous les jours


array([ 7.,  5., nan,  6.,  1.,  4.,  2.,  3.])

count    17983.000000
mean         6.250014
std          0.888699
min          1.000000
25%          6.000000
50%          6.000000
75%          7.000000
max          7.000000
Name: open_days_per_week, dtype: float64


-------------------------------------
Describe la colonne working_shifts_per_week (flexibilité horaire) :
ex : 6 jours ouverts * 2 services/jours = 12 working shifts
---------------------------------------


array([13.,  9.,  7., 12., nan, 10.,  2., 14.,  8.,  6., 11.,  4.,  5.,
        1.,  3., 15.])

...


-------------------------------------
Describe la colonne atmosphere (notes spécifiques (1 à 5) sur atmosphere) :
---------------------------------------


array([3. , nan, 3.5, 2.5, 4.5, 4. , 2. , 1.5, 5. , 1. ])

count    8246.000000
mean        3.766008
std         0.535208
min         1.000000
25%         3.500000
50%         4.000000
75%         4.000000
max         5.000000
Name: atmosphere, dtype: float64


-------------------------------------
Les valeurs manquantes des colonnes Dtype float64 (sur 31271 lignes) :
---------------------------------------


,missing_count,missing_pct
atmosphere,23025,73.63
working_shifts_per_week,13288,42.49
open_days_per_week,13288,42.49
open_hours_per_week,13288,42.49
food,12864,41.14
value,12818,40.99
service,12780,40.87
avg_rating,3034,9.70
terrible,3013,9.64
average,3013,9.64


### 2.3. Les colonnes Dtype Object

In [39]:
print("\n-------------------------------------")
print("Les valeurs uniques de la colonne claimed (Restaurant revendiqué par le propriétaire) :")
print("---------------------------------------")
display(df_trip_idf["claimed"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne awards (Récompenses TripAdvisor (Certificate of Excellence, Travellers’ Choice…)) :")
print("---------------------------------------")
display(df_trip_idf["awards"].unique()[:10])

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne popularity_detailed (Classement local précis) :")
print("---------------------------------------")
display(df_trip_idf["popularity_detailed"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne popularity_generic (Classement global “places to eat”) :")
print("---------------------------------------")
display(df_trip_idf["popularity_generic"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne top_tags (Combinaison de gamme + cuisines) :")
print("---------------------------------------")
display(df_trip_idf["top_tags"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne price_level (Indication simple du prix) :")
print("---------------------------------------")
display(df_trip_idf["price_level"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne price_range (Fourchette détaillée) :")
print("---------------------------------------")
display(df_trip_idf["price_range"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne meals (Types de repas (Breakfast, Lunch, Dinner…)) :")
print("---------------------------------------")
display(df_trip_idf["meals"].unique()[:30])

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne cuisines (Types de cuisines (liste)) :")
print("---------------------------------------")
display(df_trip_idf["cuisines"].unique()[:30])

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne special_diets (Régimes spéciaux (Vegan, Halal…)) :")
print("---------------------------------------")
display(df_trip_idf["special_diets"].unique()[:20])

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne features (Services proposés (wifi, terrasse, réservation…)) :")
print("---------------------------------------")
display(df_trip_idf["features"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne vegetarian_friendly (Indicateurs binaires (Y / N) sur label vegetarian_friendly) :")
print("---------------------------------------")
display(df_trip_idf["vegetarian_friendly"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne vegan_options (Indicateurs binaires (Y / N) sur label vegan_options) :")
print("---------------------------------------")
display(df_trip_idf["vegan_options"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne gluten_free (Indicateurs binaires (Y / N) sur label gluten_free) :")
print("---------------------------------------")
display(df_trip_idf["gluten_free"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne original_open_hours (Horaires détaillés (JSON encodé en texte)) :")
print("---------------------------------------")
display(df_trip_idf["original_open_hours"].unique()[:5])

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne default_language :")
print("---------------------------------------")
display(df_trip_idf["default_language"].unique())

print("\n-------------------------------------")
print("Les valeurs uniques de la colonne keywords (Mots-clés extraits des avis) :")
print("---------------------------------------")
display(df_trip_idf["keywords"].unique())

#Vérifier les valeurs manquantes des colonnes Dtype OBJECT
print("\n-------------------------------------")
print("Les valeurs manquantes des colonnes Dtype OBJECT (sur 31271 lignes) :")
print("---------------------------------------")
missing_object_summary = pd.DataFrame({"missing_count": df_trip_idf.select_dtypes(include="object").isna().sum(),
                                        "missing_pct": (df_trip_idf.select_dtypes(include="object").isna().sum() / 31271 * 100).round(2)}
                                      ).sort_values("missing_pct", ascending=False)
display(missing_object_summary)


-------------------------------------
Les valeurs uniques de la colonne claimed (Restaurant revendiqué par le propriétaire) :
---------------------------------------


array(['Claimed', 'Unclaimed', None], dtype=object)


-------------------------------------
Les valeurs uniques de la colonne awards (Récompenses TripAdvisor (Certificate of Excellence, Travellers’ Choice…)) :
---------------------------------------


array(["Travellers' Choice, Certificate of Excellence 2020, Certificate of Excellence 2019",
       "Travellers' Choice, Certificate of Excellence 2020, Certificate of Excellence 2019, Certificate of Excellence 2018, Certificate of Excellence 2017",
       'Certificate of Excellence 2016, Certificate of Excellence 2015',
       None,
       'Certificate of Excellence 2018, Certificate of Excellence 2016',
       'Certificate of Excellence 2015',
       "Travellers' Choice, Certificate of Excellence 2020, Certificate of Excellence 2019, Certificate of Excellence 2018, Certificate of Excellence 2017, Certificate of Excellence 2016, Certificate of Excellence 2015",
       'Certificate of Excellence 2017, Certificate of Excellence 2016',
       'Certificate of Excellence 2019, Certificate of Excellence 2018, Certificate of Excellence 2017, Certificate of Excellence 2016, Certificate of Excellence 2015',
       'Certificate of Excellence 2017, Certificate of Excellence 2016, Certificate of 


-------------------------------------
Les valeurs uniques de la colonne popularity_detailed (Classement local précis) :
---------------------------------------


...

array(['English', 'All languages', None], dtype=object)


-------------------------------------
Les valeurs uniques de la colonne keywords (Mots-clés extraits des avis) :
---------------------------------------


array([None, 'risotto, salad, antipasti, disney village, fresh food',
       'burger, potatoes, disney village, free refills, large portion of chips',
       ..., 'crepes, ice cream, delicious food, moulin, hotel',
       'indian food, quantity and quality, owner, home, paris',
       'steak, beer, paris, sign, brasserie'], shape=(1221,), dtype=object)


-------------------------------------
Les valeurs manquantes des colonnes Dtype OBJECT (sur 31271 lignes) :
---------------------------------------


,missing_count,missing_pct
keywords,30050,96.10
special_diets,24797,79.30
awards,24053,76.92
price_range,20948,66.99
features,20089,64.24
province,18138,58.00
original_open_hours,13288,42.49
meals,12052,38.54
price_level,7372,23.57
cuisines,5915,18.92


# 3. Les colonnes à prendre ou à laisser

### Les colonnes à garder abslument  :

**Dtype float**
- Itinéraire carte : latitude (0,5% missing), longitude (0,5% missing)

- Qualité globale : avg_rating (9,70% missing)

- Crédibilité: total_reviews_count (6,59% missing)


**Dtype object** :
- Filtres localisations géographiques : country (0% missing), region (0% missing), city (0,01% missing), address (0% missing)
  
- Identité : restaurant_name (0% missing), restaurant_link (0% missing)

- Statut + informations générales : claimed (0.11% missing), price_level (23.57% missing), price_range (66.99% missing), meals (38.54% missing), top_tags (12.81% missing), original_open_hours (42,49% missing)

- Labels alimentaires simples : vegetarian_friendly (0% missing), vegan_options (0% missing), gluten_free (0% missing)


### Les colonnes à transformer  :

- A transformer en score binaire (0/1) : awards (76.92% missing) 

- A transformer en proportions (%) : excellent (9.64% missing), very_good (9.64% missing), average (9.64%missing)

- A transformer en score moyen partiel : food (41.14% missing), service (40.87% missing), value (40.99% missing)

- A transformer en score d’accessibilité/Disponibilité/Flexibilité: open_days_per_week (42.49% missing)+ working_shifts_per_week (42.49% missing)

- A extraire l'option Halal de la colonne special_diets (79.30% missing) et les transformer en booléan

- A transformer en booléan : "vegetarian_friendly", "vegan_options", "gluten_free"


### Les colonnes à ignorer :
- Beaucoup de valeurs manquantes Dtype Float : atmosphere (74 % missing), open_hours_per_week (42,49% missing)

- Beaucoup de valeurs manquantes Dtype Object : keywords (96%), province (58% missing)

- Bruit: cuisines (18.92% missing), popularity_detailed (9.57% missing), popularity_generic (9.75% missing), original_location (0% missing), reviews_count_in_default_language (9.64% missing), poor (9.64% missing), terrible (9.64% missing), features (64.24% missing), default_language (9.64% missing)

# 4. Nettoyer + compléter + transformer les valeurs

### 4.1. Nettoyer

In [6]:
df_trip_idf_clean = df_trip_idf.copy()

#Supprimer toutes les lignes où latitude OU longitude est manquante (NaN)
df_trip_idf_clean = df_trip_idf_clean.dropna(subset=["latitude", "longitude"]).copy()

### 4.2. Compléter

In [7]:
#Créer une variable indicatrice pour avg_rating dont valeurs manquantes et la compléter avec une valeur neutre pondérée (median) dans une colonne à part pour l'étape score
df_trip_idf_clean["avg_rating_missing"] = (df_trip_idf_clean["avg_rating"].isna()).astype(int)
median_rating = df_trip_idf_clean["avg_rating"].median()
df_trip_idf_clean["avg_rating_filled"] = (df_trip_idf_clean["avg_rating"].fillna(median_rating))

#Créer une variable indicatrice pour total_reviews_count dont valeurs manquantes et la compléter = 0
df_trip_idf_clean["total_reviews_count_missing"] = (df_trip_idf_clean["total_reviews_count"].isna()).astype(int)
df_trip_idf_clean["total_reviews_count_filled"] = df_trip_idf_clean["total_reviews_count"].fillna(0)

#Hypothèse raisonnable : un restaurant non revendiqué n’a probablement pas fait la démarche
df_trip_idf_clean["claimed"] = df_trip_idf_clean["claimed"].fillna("Unclaimed")

### 4.3. Transformer

#### 4.3.1 Colonne "awards" --> score binaire (0/1)

In [9]:
#transformer en score binaire (0/1) : awards (76.92% missing)
df_trip_idf_clean["awards_binary"] = (df_trip_idf_clean["awards"].notna()).astype(int)
df_trip_idf_clean[["awards", "awards_binary"]].head(5)

,awards,awards_binary
74,"Travellers' Choice, Certificate of Excellence ...",1
75,"Travellers' Choice, Certificate of Excellence ...",1
76,"Certificate of Excellence 2016, Certificate of...",1
77,None,0
78,None,0


#### 4.3.2 Colonnes "excellent", "very_good", "average" --> proportions (%)

In [10]:
#transformer en proportions (%) : excellent (9.64% missing), very_good (9.64% missing), average (9.64%missing)

#compléter les valeurs manquantes des colonnes "excellent", "very_good", "average" = 0
df_trip_idf_clean[["excellent", "very_good", "average"]] = df_trip_idf_clean[["excellent", "very_good", "average"]].fillna(0)

#les transformer en proportions (%) en basant sur la colonne "total_reviews_count"
df_trip_idf_clean["excellent_pct"] = ((df_trip_idf_clean["excellent"] / df_trip_idf_clean["total_reviews_count"])*100).round(2)
df_trip_idf_clean["very_good_pct"] = ((df_trip_idf_clean["very_good"] / df_trip_idf_clean["total_reviews_count"])*100).round(2)
df_trip_idf_clean["average_pct"] = ((df_trip_idf_clean["average"] / df_trip_idf_clean["total_reviews_count"])*100).round(2)

#vérifier
df_trip_idf_clean[["total_reviews_count","excellent","excellent_pct", "very_good", "very_good_pct", "average","average_pct"]].head(5)

,total_reviews_count,excellent,excellent_pct,very_good,very_good_pct,average,average_pct
74,292.0,1.0,0.34,1.0,0.34,0.0,0.00
75,296.0,202.0,68.24,73.0,24.66,18.0,6.08
76,204.0,3.0,1.47,2.0,0.98,1.0,0.49
77,17.0,5.0,29.41,5.0,29.41,2.0,11.76
78,12.0,0.0,0.00,1.0,8.33,0.0,0.00


#### 4.3.3 Colonnes "food", "service", "value" --> score moyen partiel

In [11]:
#transformer en score moyen partiel : food (41.14% missing), service (40.87% missing), value (40.99% missing)

#Créer le score moyen partiel en ignorant automatiquement les NaN + caculant la moyenne sur 1, 2 ou 3 colonnes + retournant NaN seulement si les 3 sont manquantes
df_trip_idf_clean["partial_quality_score"] = ((df_trip_idf_clean[["food", "service", "value"]].mean(axis=1, skipna=True))).round(2)

#Ajouter un indicateur de fiabilité (3=très fiable grace aux valeurs présentes sur les 3 colonnes, 2=fiable, 1=faible, 0=aucune info)
df_trip_idf_clean["partial_quality_count"] = (df_trip_idf_clean[["food", "service", "value"]].notna().sum(axis=1))

#Flager score partiel incomplet (inférieur à 3)
df_trip_idf_clean["partial_quality_missing"] = (df_trip_idf_clean["partial_quality_count"] < 3).astype(int)

#Score pondéré par la fiabilité
df_trip_idf_clean["partial_quality_score_weighted"] = (df_trip_idf_clean["partial_quality_score"] * (df_trip_idf_clean["partial_quality_count"] / 3)).round(2)

#vérifier
df_trip_idf_clean[["food", "service", "value", "partial_quality_score", "partial_quality_count", "partial_quality_score_weighted"]].head(5)

,food,service,value,partial_quality_score,partial_quality_count,partial_quality_score_weighted
74,4.0,4.0,3.5,3.83,3,3.83
75,4.5,4.5,4.5,4.50,3,4.50
76,4.0,4.0,3.5,3.83,3,3.83
77,4.0,3.0,4.0,3.67,3,3.67
78,NaN,NaN,NaN,NaN,0,NaN


#### 4.3.4 Colonnes "open_days_per_week", "working_shifts_per_week" --> score d’accessibilité

In [12]:
#transformer en score d’accessibilité/Disponibilité/Flexibilité: open_days_per_week (42.49% missing)+ working_shifts_per_week (42.49% missing)

#Créer un indicateurs sur les valeurs manquantes
df_trip_idf_clean["open_days_missing"] = (df_trip_idf_clean["open_days_per_week"].isna().astype(int))
df_trip_idf_clean["working_shifts_missing"] = (df_trip_idf_clean["working_shifts_per_week"].isna().astype(int))

#Normaliser les deux variables (0 → 1)
df_trip_idf_clean["open_days_norm"] = ((df_trip_idf_clean["open_days_per_week"] / 7)).round(2)
df_trip_idf_clean["working_shifts_norm"] = ((df_trip_idf_clean["working_shifts_per_week"] / 14)).round(2)

#Score d’accessibilité partiel
df_trip_idf_clean["accessibility_score"] = ((df_trip_idf_clean[["open_days_norm", "working_shifts_norm"]].mean(axis=1, skipna=True))).round(2)

#Fiabilité du score : 2=très fiable, 1=partiel, 0=aucune info
df_trip_idf_clean["accessibility_count"] = (df_trip_idf_clean[["open_days_norm", "working_shifts_norm"]].notna().sum(axis=1))

#Flag “accessibilité incomplète”
df_trip_idf_clean["accessibility_missing"] = (df_trip_idf_clean["accessibility_count"] < 2).astype(int)

#Mise à l’échelle finale
df_trip_idf_clean["accessibility_score_5"] = (df_trip_idf_clean["accessibility_score"] * 5).round(2)

#vérifier
df_trip_idf_clean[["open_days_per_week", "working_shifts_per_week", "open_days_norm", "working_shifts_norm",
                 "accessibility_score", "accessibility_count", "accessibility_score_5"]].head(5)

,open_days_per_week,working_shifts_per_week,open_days_norm,working_shifts_norm,accessibility_score,accessibility_count,accessibility_score_5
74,7.0,13.0,1.00,0.93,0.97,2,4.85
75,5.0,9.0,0.71,0.64,0.68,2,3.40
76,7.0,7.0,1.00,0.50,0.75,2,3.75
77,7.0,12.0,1.00,0.86,0.93,2,4.65
78,NaN,NaN,NaN,NaN,NaN,0,NaN


#### 4.3.5 Extraire l'option Halal & kosher de la colonne special_diets (79.30% missing) --> booléenne

In [13]:
# Création de la colonne booléenne halal
df_trip_idf_clean['is_halal'] = df_trip_idf_clean['special_diets'].str.contains('Halal', case=False, na=False)
# Vérification
display(df_trip_idf_clean['is_halal'].value_counts())

# Création de la colonne booléenne kosher
df_trip_idf_clean['is_kosher'] = df_trip_idf_clean['special_diets'].str.contains('kosher', case=False, na=False)
# Vérification
display(df_trip_idf_clean['is_kosher'].value_counts())

is_halal
False    30831
True       267
Name: count, dtype: int64

is_kosher
False    31030
True        68
Name: count, dtype: int64

#### 4.3.6 Transformer en booléan : "vegetarian_friendly", "vegan_options", "gluten_free" --> créer une colonne "diet_level" pour UX streamlit

In [14]:
#Transformation directe Y → True, N → False
cols = ["vegetarian_friendly", "vegan_options", "gluten_free"]
df_trip_idf_clean[cols] = (df_trip_idf_clean[cols].replace({'Y': True, 'N': False}).astype(bool))

#créer une colonne "diet_level" pour UX streamlit
df_trip_idf_clean["diet_level"] = np.select([df_trip_idf_clean["vegan_options"] == True,
                                             df_trip_idf_clean["vegetarian_friendly"] == True,],
                                            ["vegan", "vegetarian"],
                                            default="none")

#vérifier
df_trip_idf_clean[["vegetarian_friendly", "vegan_options", "diet_level"]]

C:\Users\DELL\AppData\Local\Temp\ipykernel_17008\2730700687.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_trip_idf_clean[cols] = (df_trip_idf_clean[cols].replace({'Y': True, 'N': False}).astype(bool))


,vegetarian_friendly,vegan_options,diet_level
74,False,False,none
75,True,False,vegetarian
76,True,False,vegetarian
77,False,False,none
78,False,False,none
...,...,...,...
154861,False,False,none
155067,False,False,none
155256,True,False,vegetarian
155286,False,False,none


#### 4.3.7 Créer une colonne url en basant sur la colonne "restaurant_link"

In [15]:
#Créer une colonne url en basant sur la colonne "restaurant_link"
df_trip_idf_clean["url"] = ("https://www.tripadvisor.fr/Restaurant_Review-" + df_trip_idf_clean["restaurant_link"].astype(str) + ".html")
#vérifier
df_trip_idf_clean[["restaurant_link", "url"]]

,restaurant_link,url
74,g1006520-d1007522,https://www.tripadvisor.fr/Restaurant_Review-g...
75,g1006520-d10128445,https://www.tripadvisor.fr/Restaurant_Review-g...
76,g1006520-d1017687,https://www.tripadvisor.fr/Restaurant_Review-g...
77,g1006520-d10537685,https://www.tripadvisor.fr/Restaurant_Review-g...
78,g1006520-d10796472,https://www.tripadvisor.fr/Restaurant_Review-g...
...,...,...
154861,g947976-d4148466,https://www.tripadvisor.fr/Restaurant_Review-g...
155067,g9565875-d6695339,https://www.tripadvisor.fr/Restaurant_Review-g...
155256,g9756852-d10210681,https://www.tripadvisor.fr/Restaurant_Review-g...
155286,g9980751-d16852973,https://www.tripadvisor.fr/Restaurant_Review-g...


#### 4.3.8 catégoriser la colonne "price_level" en petit (1), moyen (2) et confort (3) budget

In [16]:
#Créer une catégorie explicite Unknown pour les price_level dont valeurs manquantes
df_trip_idf_clean["price_level"] = df_trip_idf_clean["price_level"].fillna("Unknown")

#mapper les valeurs
price_mapping = {"€": 1, "€€-€€€": 2, "€€€€": 3}
df_trip_idf_clean["price_category"] = (df_trip_idf_clean["price_level"].map(price_mapping).fillna(1))

#vérifier
df_trip_idf_clean["price_category"].value_counts()

price_category
2.0    16882
1.0    13067
3.0     1149
Name: count, dtype: int64

#### 4.3.9 transformer la colonne "meals" en tags normalisés + colonnes booléennes.

In [17]:
# DEF : Normaliser + tokenizer meals
def parse_meals(x):
    if pd.isna(x) or str(x).strip() == "":
        return []
    # split + trim + normalisation simple
    items = [t.strip() for t in str(x).split(",")]
    # remove empty + unify casing
    items = [t for t in items if t]
    return items

df_trip_idf_clean["meals_list"] = df_trip_idf_clean["meals"].apply(parse_meals)

# liste de tags : Créer des colonnes booléennes (idéal pour filtres rapides sur streamlit)
MEAL_TAGS = ["Breakfast", "Brunch", "Lunch", "Dinner", "After-hours", "Drinks"]
for tag in MEAL_TAGS:
    col = "has_" + tag.lower().replace("-", "_")
    df_trip_idf_clean[col] = df_trip_idf_clean["meals_list"].apply(lambda lst: tag in lst)

#vérifier
df_trip_idf_clean[["meals_list", "has_breakfast", "has_brunch", "has_lunch", "has_dinner", "has_after_hours", "has_drinks"]].head()

,meals_list,has_breakfast,has_brunch,has_lunch,has_dinner,has_after_hours,has_drinks
74,"[Lunch, Dinner, After-hours]",False,False,True,True,True,False
75,"[Lunch, Dinner]",False,False,True,True,False,False
76,"[Lunch, Dinner]",False,False,True,True,False,False
77,"[Lunch, Dinner]",False,False,True,True,False,False
78,[Dinner],False,False,False,True,False,False


#### 4.3.10 transformer la colonne "cuisines" en tags normalisés + colonnes booléennes.

In [18]:
## DEF : Normaliser + tokenizer meals
def parse_cuisines(x):
    if pd.isna(x) or str(x).strip() == "" or str(x).strip().lower() == "none":
        return []
    items = [t.strip() for t in str(x).split(",")]
    items = [t for t in items if t]
    # Normalisation simple (optionnel)
    # ex: uniformiser "Fast food" vs "Fast food " etc.
    return items

df_trip_idf_clean["cuisines_list"] = df_trip_idf_clean["cuisines"].apply(parse_cuisines)

display(df_trip_idf_clean["cuisines_list"])

# count les mots
cuisine_word_counts = (df_trip_idf_clean["cuisines_list"].explode().value_counts().reset_index())
cuisine_word_counts.columns = ["cuisine", "count"]
#pd.set_option("display.max_rows", None)
display(cuisine_word_counts)

74                                     [French]
75                                     [French]
76                     [Japanese, Sushi, Asian]
77                     [Japanese, Sushi, Asian]
78                  [Pakistani, Middle Eastern]
                          ...                  
154861                                    [Pub]
155067                          [French, Diner]
155256    [French, American, Street Food, Deli]
155286                               [European]
155287                                 [French]
Name: cuisines_list, Length: 31098, dtype: object

,cuisine,count
0,French,11615
1,European,5637
2,Asian,3769
3,Italian,3304
4,Japanese,2048
...,...,...
136,Croatian,1
137,Hokkaido cuisine,1
138,Kyoto cuisine,1
139,Bahamian,1


In [20]:
CUISINE_CONTINENT_MAP = {
    "French": "European",
    "Italian": "European",
    "Southern-Italian": "European",
    "Central-Italian": "European",
    "Northern-Italian": "European",
    "Neapolitan": "European",
    "Sicilian": "European",
    "Sardinian": "European",
    "Tuscan": "European",
    "Campania": "European",
    "Lazio": "European",
    "Romana": "European",
    "Apulian": "European",
    "Calabrian": "European",
    "Romagna": "European",
    "Spanish": "European",
    "Greek": "European",
    "Portuguese": "European",
    "Belgian": "European",
    "British": "European",
    "Irish": "European",
    "Scottish": "European",
    "Welsh": "European",
    "German": "European",
    "Swiss": "European",
    "Austrian": "European",
    "Dutch": "European",
    "Scandinavian": "European",
    "Swedish": "European",
    "Norwegian": "European",
    "Danish": "European",
    "Polish": "European",
    "Hungarian": "European",
    "Romanian": "European",
    "Croatian": "European",
    "Albanian": "European",
    "Ukrainian": "European",
    "Russian": "European",
    "Central European": "European",
    "Eastern European": "European",
    "Catalan": "European",

    "Asian": "Asian",
    "Japanese": "Asian",
    "Japanese Fusion": "Asian",
    "Chinese": "Asian",
    "Beijing cuisine": "Asian",
    "Fujian": "Asian",
    "Xinjiang": "Asian",
    "Yunnan": "Asian",
    "Korean": "Asian",
    "Vietnamese": "Asian",
    "Thai": "Asian",
    "Indian": "Asian",
    "Pakistani": "Asian",
    "Bangladeshi": "Asian",
    "Sri Lankan": "Asian",
    "Nepalese": "Asian",
    "Tibetan": "Asian",
    "Indonesian": "Asian",
    "Malaysian": "Asian",
    "Singaporean": "Asian",
    "Taiwanese": "Asian",
    "Philippine": "Asian",
    "Mongolian": "Asian",
    "Central Asian": "Asian",
    "Uzbek": "Asian",
    "Afghani": "Asian",


    "Middle Eastern": "Middle Eastern",
    "Lebanese": "Middle Eastern",
    "Israeli": "Middle Eastern",
    "Persian": "Middle Eastern",
    "Arabic": "Middle Eastern",
    "Armenian": "Middle Eastern",
    "Assyrian": "Middle Eastern",


    "African": "African",
    "Moroccan": "African",
    "Tunisian": "African",
    "Algerian": "African",
    "Egyptian": "African",
    "Ethiopian": "African",
    "Nigerian": "African",


    "American": "American",
    "Mexican": "American",
    "South American": "American",
    "Brazilian": "American",
    "Argentinian": "American",
    "Peruvian": "American",
    "Colombian": "American",
    "Chilean": "American",
    "Venezuelan": "American",
    "Canadian": "American",
    "Native American": "American",
    "Southwestern": "American",


    "Caribbean": "Caribbean",
    "Jamaican": "Caribbean",
    "Cuban": "Caribbean",
    "Puerto Rican": "Caribbean",
    "Bahamian": "Caribbean",


    "International": "International",
    "Fusion": "International",
    "Contemporary": "International",
    "Medicinal foods": "International",
    "Caucasian": "International",}

CUISINE_TYPE_MAP = {
    "Fast food": "Fast food",
    "Street Food": "Street Food",
    "Pizza": "Pizza",
    "Diner": "Diner",

    "Bar": "Bar",
    "Pub": "Bar",
    "Wine Bar": "Bar",
    "Brew Pub": "Bar",
    "Dining bars": "Bar",
    "Beer restaurants": "Bar",

    "Cafe": "Cafe",
    "Deli": "Deli",

    "Healthy": "Healthy",
    "Soups": "Healthy",
    "Medicinal foods": "Healthy",

    "Grill": "Grill",
    "Barbecue": "Grill",
    "Steakhouse": "Steakhouse",

    "Seafood": "Seafood",
    "Sushi": "Sushi",
    "Japanese sweets parlour": "Sweets",
    "Fruit parlours": "Sweets",

    "Gastropub": "Gastropub",
    "Contemporary": "Gastronomic",
    "Fusion": "Fusion",}

#Fonctions
def get_cuisine_continent(cuisines_list):
    for c in cuisines_list:
        if c in CUISINE_CONTINENT_MAP:
            return CUISINE_CONTINENT_MAP[c]
    return "Other"


def get_cuisine_type(cuisines_list):
    for c in cuisines_list:
        if c in CUISINE_TYPE_MAP:
            return CUISINE_TYPE_MAP[c]
    return "Other"

#apply
df_trip_idf_clean["cuisine_continent"] = df_trip_idf_clean["cuisines_list"].apply(get_cuisine_continent)
df_trip_idf_clean["cuisine_type"] = df_trip_idf_clean["cuisines_list"].apply(get_cuisine_type)

df_trip_idf_clean[["cuisines_list", "cuisine_continent", "cuisine_type"]]

,cuisines_list,cuisine_continent,cuisine_type
74,[French],European,Other
75,[French],European,Other
76,"[Japanese, Sushi, Asian]",Asian,Sushi
77,"[Japanese, Sushi, Asian]",Asian,Sushi
78,"[Pakistani, Middle Eastern]",Asian,Other
...,...,...,...
154861,[Pub],Other,Bar
155067,"[French, Diner]",European,Diner
155256,"[French, American, Street Food, Deli]",European,Street Food
155286,[European],Other,Other


#### 4.3.11 Calculer un score final en basant sur 4 catégories scores --> formule : RecoScore=100×(0.40⋅Q+0.25⋅E+0.20⋅R+0.15⋅A)

Q = (40 %) qualité globale (rating) = "avg_rating_filled",

E = (25 %) excellence (avis “Excellent”) = "excellent_pct",

R = (20 %) fiabilité (volume d’avis) = "total_reviews_count_filled",

A = (15 %) accessibilité (horaires) = "accessibility_score_5"

Lecture du score : 85 – 100 = Exceptionnel ; 70 – 85 = Très recommandé ; 55 – 70 = Correct ; < 55	= Peu recommandé

In [19]:
#1) Normalisations
# Rating global
df_trip_idf_clean["Q_rating"] = df_trip_idf_clean["avg_rating_filled"] / 5

# Qualité fine (food / service / value)
df_trip_idf_clean["Q_partial"] = (df_trip_idf_clean["partial_quality_score_weighted"].fillna(df_trip_idf_clean["partial_quality_score_weighted"].median()) / 5)

# Excellence
df_trip_idf_clean["E"] = df_trip_idf_clean["excellent_pct"].fillna(0) / 100

# Fiabilité
df_trip_idf_clean["R"] = np.log1p(df_trip_idf_clean["total_reviews_count_filled"]) / np.log1p(df_trip_idf_clean["total_reviews_count_filled"].max())

# Accessibilité
df_trip_idf_clean["A"] = (df_trip_idf_clean["accessibility_score_5"].fillna(df_trip_idf_clean["accessibility_score_5"].median()) / 5)

# 2) Qualité globale (Q combinée) : combiner rating global (70%) + qualité fine (30%) :
df_trip_idf_clean["Q"] = 0.7 * df_trip_idf_clean["Q_rating"] + 0.3 * df_trip_idf_clean["Q_partial"]

# 3) Score final de recommandation streamlit
df_trip_idf_clean["reco_score"] = 100 * (0.40 * df_trip_idf_clean["Q"] + 0.25 * df_trip_idf_clean["E"] + 0.20 * df_trip_idf_clean["R"] + 0.15 * df_trip_idf_clean["A"]) + (df_trip_idf_clean["awards_binary"] * 3)

df_trip_idf_clean[["Q_rating", "Q_partial", "Q", "E", "R", "reco_score"]]

,Q_rating,Q_partial,Q,E,R,reco_score
74,0.8,0.766,0.7898,0.0034,0.589944,61.025878
75,0.9,0.900,0.9000,0.6824,0.591352,78.087044
76,0.8,0.766,0.7898,0.0147,0.552849,57.266478
77,0.7,0.734,0.7102,0.2941,0.300195,55.714392
78,0.7,0.800,0.7300,0.0000,0.266396,45.777923
...,...,...,...,...,...,...
154861,0.8,0.800,0.8000,0.5000,0.114102,58.032042
155067,0.9,0.834,0.8802,0.4725,0.469634,64.963178
155256,1.0,1.000,1.0000,0.9265,0.439755,85.607603
155286,0.8,0.800,0.8000,0.0000,0.000000,43.250000


#### 4.3.12 Créer une colonne "postal_code" par extraire la colonne "adresse"

In [20]:
#extraire code postal de la colonne "adresse"
df_trip_idf_clean["postal_code"] = (df_trip_idf_clean["address"].str.extract(r"(75\d{3}|77\d{3}|78\d{3}|91\d{3}|92\d{3}|93\d{3}|94\d{3}|95\d{3})"))
df_trip_idf_clean[["address", "postal_code"]]

,address,postal_code
74,"9 quai de la Republique, 78700 Conflans Sainte...",78700
75,"91 avenue Carnot, 78700 Conflans Sainte Honori...",78700
76,"2 boulevard Troussel, 78700 Conflans Sainte Ho...",78700
77,"127 rue Desire Clement, 78700 Conflans Sainte ...",78700
78,"133 rue Désiré Clément Restaurant Ouvert 7/7, ...",78700
...,...,...
154861,"17 rue des Marais, 77440 Congis-sur-Therouanne...",77440
155067,"34 Grande rue, 78770 Auteuil-le-Roi France",78770
155256,"25 rue des Pres, 78790 Arnouville-les-Mantes F...",78790
155286,"14 route de Flandres, 95500 Bonneuil-en-France...",95500


# 5. DataFrame Final (normer les colonnes afin de mapper avec les autres datasets dans streamlit)

19 colonnes normées UX streamlit : source_id, source, type, raw_type, url, lat, lon, name, address, postal_code, city, region, country, snippet, rating, review_count, price_level, max_people, distance_km

In [25]:
# Garder uniquement les colonnes utiles
cols_keep = ["restaurant_link", "restaurant_name", "country", "region", "city", "address", "latitude", "longitude", "postal_code", "avg_rating_filled", "total_reviews_count_filled", "url", "price_category", 
             "cuisines_list", "gluten_free", "awards_binary", "excellent_pct", "partial_quality_score_weighted", "accessibility_score_5", "is_halal", "is_kosher",
             "diet_level", "has_breakfast", "has_brunch", "has_lunch", "has_dinner", "has_after_hours", "has_drinks", "reco_score"]
df_trip_idf_clean_norm = df_trip_idf_clean[cols_keep].copy()

# Colonnes standard (type + distance_km placeholder)
df_trip_idf_clean_norm["source"] = "tripadvisor_european_restaurants"
df_trip_idf_clean_norm["type"] = "Restaurant"
df_trip_idf_clean_norm["raw_type"] = np.nan
df_trip_idf_clean_norm["distance_km"] = np.nan
df_trip_idf_clean_norm["max_people"] = np.nan

# Normaliser / renommer

rename_map={"restaurant_link": "source_id",
            "restaurant_name": "name",
            "latitude": "lat",
            "longitude": "lon",
            "cuisines_list": " snippet",
            "price_category": "price_level",
            "avg_rating_filled": "rating",
            "total_reviews_count_filled": "review_count"}
df_trip_idf_clean_normux = df_trip_idf_clean_norm.rename(columns = rename_map, errors="ignore")

# réordonner les colonnes UX
UX_COLUMNS = ["source_id", "source", "type", "raw_type", "url", "lat", "lon",
              "name", "address", "postal_code", "city", "region", "country",
              "snippet", "rating", "review_count", "price_level", "max_people", "distance_km",
              "awards_binary", "diet_level", "gluten_free", "is_halal", "is_kosher",
              "has_breakfast", "has_brunch", "has_lunch", "has_dinner", "has_after_hours", "has_drinks"
              "excellent_pct", "partial_quality_score_weighted", "accessibility_score_5", "reco_score"]

for col in UX_COLUMNS:
    if col not in df_trip_idf_clean_normux.columns:
        df_trip_idf_clean_normux[col] = None
        
df_trip_idf_clean_normux_ = df_trip_idf_clean_normux[UX_COLUMNS]

#sauvegarder les changements
df_airbnb_clean_normux_fast = df_trip_idf_clean_normux_.copy()

# Export parquet final (un seul fichier)
df_airbnb_clean_normux_fast.to_parquet("df_airbnb_clean_normux_fast.parquet", index=False)

display(df_airbnb_clean_normux_fast.head(2))
display(df_airbnb_clean_normux_fast.info())

#Exporter en CSV
#output_path = "df_trip_idf_clean_final.csv"
#df_trip_idf_clean_final.to_csv(output_path, index = False, encoding="utf-8-sig")

,source_id,source,type,raw_type,url,lat,lon,name,address,postal_code,city,region,country,snippet,rating,review_count,price_level,max_people,distance_km,awards_binary,diet_level,gluten_free,is_halal,is_kosher,has_breakfast,has_brunch,has_lunch,has_dinner,has_after_hours,has_drinksexcellent_pct,partial_quality_score_weighted,accessibility_score_5,reco_score
74,g1006520-d1007522,tripadvisor_european_restaurants,Restaurant,NaN,https://www.tripadvisor.fr/Restaurant_Review-g...,48.99111,2.088753,L'imprevu,"9 quai de la Republique, 78700 Conflans Sainte...",78700,Conflans Sainte Honorine,Ile-de-France,France,None,4.0,292.0,3.0,NaN,NaN,1,none,False,False,False,False,False,True,True,True,None,3.83,4.85,61.025878
75,g1006520-d10128445,tripadvisor_european_restaurants,Restaurant,NaN,https://www.tripadvisor.fr/Restaurant_Review-g...,49.00057,2.093569,La Maison du Poulet,"91 avenue Carnot, 78700 Conflans Sainte Honori...",78700,Conflans Sainte Honorine,Ile-de-France,France,None,4.5,296.0,2.0,NaN,NaN,1,vegetarian,False,False,False,False,False,True,True,False,None,4.50,3.40,78.087044


<class 'pandas.core.frame.DataFrame'>
Index: 31098 entries, 74 to 155287
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   source_id                       31098 non-null  object 
 1   source                          31098 non-null  object 
 2   type                            31098 non-null  object 
 3   raw_type                        0 non-null      float64
 4   url                             31098 non-null  object 
 5   lat                             31098 non-null  float64
 6   lon                             31098 non-null  float64
 7   name                            31098 non-null  object 
 8   address                         31098 non-null  object 
 9   postal_code                     30635 non-null  object 
 10  city                            31094 non-null  object 
 11  region                          31098 non-null  object 
 12  country                         310

None

# 6. Carte de points avec clustering

In [39]:
#Créer la carte
#m = folium.Map(location=[48.85, 2.35], zoom_start=9, tiles="OpenStreetMap")

#Ajouter le clustering
#marker_cluster = MarkerCluster().add_to(m)

#Ajouter les POI
#for _, row in df_trip_idf_clean_final.iterrows():
#    folium.Marker(location=[row["latitude"], row["longitude"]], popup=f"""<b>{row['restaurant_name']}</b><br><br><i>{row['top_tags']}<i><br><br>{row['address']}""").add_to(marker_cluster)

#m.save("map_tripadvisor_idf.html")
#m